In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression

In [12]:
PATH = 'C:/input/Fast_AI/aclImdb/'
names = ['neg','pos']

In [13]:
!ls {PATH}

README
aclImdb.tar
imdb.vocab
imdbEr.txt
test
train


In [14]:
!ls {PATH}train/pos |head

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt


ls: write error


In [15]:
trn, trn_y = texts_labels_from_folders('C:/input/Fast_AI/aclImdb/train',names)
val, val_y = texts_labels_from_folders('C:/input/Fast_AI/aclImdb/test', names)
len(trn),len(val)

(25000, 25000)

In [16]:
trn[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [17]:
trn_y[0]

0

CountVectorizer converts a collection of text documents to a matrix of token counts 
(part of sklearn.feature_extraction.text).

In [18]:
veczr = CountVectorizer(tokenizer=tokenize)

fit_transform(trn) finds the vocabulary in the training set. It also transforms the training set into a term-document matrix. Since we have to apply the same transformation to your validation set, the second line uses just the method transform(val). trn_term_doc and val_term_doc are sparse matrices. trn_term_doc[i] represents training document i and it contains a count of words for each document for each word in the vocabulary.

In [42]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [43]:
trn_term_doc

<25000x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 3749745 stored elements in Compressed Sparse Row format>

In [142]:
?CountVectorizer

In [45]:
vocab = veczr.get_feature_names();vocab[5000:5005]

['aussie', 'aussies', 'austen', 'austeniana', 'austens']

In [52]:
veczr.vocabulary_['absurd'] # reverse dictionary

1297

In [51]:
trn_term_doc[0,1999]

0

2 Naive Bayes

define the log-count ratio r for each word f:
ratio of featur in positive documents is the number of times a positive document has a feature divided by the number of positive documents
r = log (ratio of feature in positive documents)/(ratio of feature in negative documents)


In [101]:
x = trn_term_doc
y = trn_y
p = x[y==1].sum(0) + 1 #正面评价的文档中，sum(0)按照0轴方向求和，+1平滑, 得到每个词频
q = x[y==0].sum(0) + 1 #负面评价的文档中，sum(0)按照0轴方向求和，+1平滑，得到每个词频
x.shape,y.shape,p.shape,q.shape,p.sum(),q.sum()
r = np.log( (p/(len(y[y==1]) + 1)) / (q/(len(y[y==0]) + 1)) ) # 两个方法效果一样，建议使用如下pr函数
b = np.log(len(p) / len(q))

In [102]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum() + 1)

In [103]:
r2 = np.log(pr(1)/pr(0))

In [106]:
print(r[0,0:5],r.shape, r2[0,0:5],r2.shape, (r==r2).mean()) 

[[ 0.69315 -0.69315 -0.07511 -0.0804  -0.18522]] (1, 75132) [[ 0.69315 -0.69315 -0.07511 -0.0804  -0.18522]] (1, 75132) 1.0


In [94]:
print((y==1).sum(), len(y[y==1]))

12500 12500


In [110]:
pre_preds = val_term_doc @ r.T + b # (25000, 1), 加b是正负样本的比例要相乘，在log中相乘变相加
preds = pre_preds.T > 0 # log值，大于0表示正面>负面
(preds == val_y).mean() # 预测为正的准确率是多少，0.81

0.81656

... and Binarized Naive Bayes

In [112]:
pre_preds = val_term_doc.sign() @ r.T + b # sign()将positive取代为1
preds = pre_preds.T > 0
(preds == val_y).mean() # 0.83

0.83184

Logistic Regression

In [134]:
print(type(x),x.shape,y.shape) # (25000, 75132) (25000,)
print(x[0].shape,type(x[0]), x[0])
print('------------------------')
print(x[0].sign())

<class 'scipy.sparse.csr.csr_matrix'> (25000, 75132) (25000,)
(1, 75132) <class 'scipy.sparse.csr.csr_matrix'>   (0, 9044)	1
  (0, 58714)	1
  (0, 10333)	1
  (0, 25638)	1
  (0, 25964)	1
  (0, 3219)	1
  (0, 36743)	1
  (0, 57324)	1
  (0, 63012)	1
  (0, 74935)	1
  (0, 71554)	1
  (0, 28558)	1
  (0, 26463)	2
  (0, 12459)	1
  (0, 28081)	1
  (0, 61729)	1
  (0, 66647)	1
  (0, 42710)	1
  (0, 74478)	1
  (0, 66418)	1
  (0, 7063)	1
  (0, 38548)	1
  (0, 65920)	1
  (0, 46986)	1
  (0, 28348)	1
  :	:
  (0, 4926)	1
  (0, 47251)	1
  (0, 25595)	1
  (0, 13479)	1
  (0, 1297)	2
  (0, 22881)	1
  (0, 66275)	1
  (0, 34616)	2
  (0, 66441)	1
  (0, 57971)	1
  (0, 47106)	1
  (0, 73488)	3
  (0, 47523)	1
  (0, 63029)	1
  (0, 15)	8
  (0, 49997)	1
  (0, 25457)	1
  (0, 24129)	1
  (0, 70041)	1
  (0, 30016)	1
  (0, 73048)	1
  (0, 40639)	1
  (0, 1050)	7
  (0, 46749)	3
  (0, 63537)	1
------------------------
  (0, 9044)	1
  (0, 58714)	1
  (0, 10333)	1
  (0, 25638)	1
  (0, 25964)	1
  (0, 3219)	1
  (0, 36743)	1
  (0, 57324)	1

In [127]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x,y)

LogisticRegression(C=100000000.0, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [135]:
preds = m.predict(val_term_doc)
(preds == val_y).mean() # 0.85更好了！

0.8334

In [136]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(trn_term_doc.sign(),y)
preds = m.predict(val_term_doc.sign())
(preds == val_y).mean()

0.85516

...and the regularzied version

In [137]:
m = LogisticRegression(C=0.1, dual=True) # c越小越regularization
m.fit(x,y)

LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [138]:
preds = m.predict(val_term_doc)
(preds == val_y).mean() # 0.88更好了！

0.88256

Trigran with NB features
use bigrams and trigrams, 2词汇和3词汇

In [140]:
veczr = CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=800000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)


In [141]:
trn_term_doc.shape

(25000, 800000)

In [143]:
vocab = veczr.get_feature_names()

In [144]:
vocab[200000:200005]

['by vast', 'by vengeance', 'by vengeance .', 'by vera', 'by vera miles']

Naive Bayes

In [146]:
x = trn_term_doc
y = trn_y
p = x[y==1].sum(0) + 1 #正面评价的文档中，sum(0)按照0轴方向求和，+1平滑, 得到每个词频
q = x[y==0].sum(0) + 1 #负面评价的文档中，sum(0)按照0轴方向求和，+1平滑，得到每个词频
x.shape,y.shape,p.shape,q.shape,p.sum(),q.sum()
r = np.log( (p/(len(y[y==1]) + 1)) / (q/(len(y[y==0]) + 1)) ) # 两个方法效果一样，建议使用如下pr函数
b = np.log(len(p) / len(q))

In [147]:
r3 = np.log(pr(1)/pr(0))

In [148]:
pre_preds = val_term_doc @ r3.T + b # (25000, 1), 加b是正负样本的比例要相乘，在log中相乘变相加
preds = pre_preds.T > 0 # log值，大于0表示原始的正面概率>负面概率
(preds == val_y).mean() # 预测为正的准确率是多少，0.87

0.86904

LogisticRegression

In [150]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x,y)
preds = m.predict(val_term_doc)
(preds == val_y).mean() # 0.90

0.90196

In [151]:
r3.shape, r3

((1, 800000),
 matrix([[-0.07511, -0.00034,  0.10548, ...,  1.38629, -2.07944, -2.07944]]))

In [155]:
np.exp(r3),r3

(matrix([[0.92764, 0.99966, 1.11125, ..., 4.     , 0.125  , 0.125  ]]),
 matrix([[-0.07511, -0.00034,  0.10548, ...,  1.38629, -2.07944, -2.07944]]))

fit regularzied logistic regression where the features are the trigrams's log-count ratios

In [161]:
print(x.shape, r3.shape) # (25000, 800000) (1, 800000)
# 内存错误，这将产生2e10个元素，每个元素32bit也要80G连续内存
x_nb = x.multiply(r3) # multiply,element-wise 
m = LogisticRegression(C=0.1, dual=True)
m.fit(x_nb,y)

val_x_nb = val_x.multiply(r3)
preds = m.predict(val_x_nb)
(preds == val_y).mean() # 0.92

(25000, 800000) (1, 800000)


MemoryError: 

fastai NBSVM++

In [162]:
# get a model from a bag of words
sl=2000 # up to 2000 words
md = TextClassifierData.from_bow(trn_term_doc, trn_y, val_term_doc, val_y, sl)

In [163]:
?TextClassifierData

In [165]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-6, cycle_len=1)

epoch      trn_loss   val_loss   <lambda>   
    0      0.024127   0.119692   0.91584   



[0.11969218035936356, 0.9158400000381469]

In [166]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

epoch      trn_loss   val_loss   <lambda>   
    0      0.02162    0.113751   0.92076   
    1      0.011307   0.111577   0.92108   



[0.11157735411286354, 0.9210800000381469]

In [1]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

NameError: name 'learner' is not defined

In [4]:
?CountVectorizer